# 🦾  Treat it as a project.
Yeah. I saw an introduction and this dataset looks amazing. This time, we are going to be dealing with a **new** dataset if the **human** activity.

This will be a new kind of problem that we will be trying to solve. Here, the dataset includes the "measurements" taken from the smart watch ⌚ from multiple people throughout the day.

> The goal is to classify **whether** a person is *sitting*, *dancing*, *sleeping* or whatever! Or just bathing in his bathtub. There will be 6 classes to classify. Hence, it is a **multiclass-classification** problem. But, with the time-series.


There are `2` sensors are used for this dataset: 
1. Gyroscope
2. Accelerometer

Each of them returns `3` separate channels because we are living in the 3D space.

___
Dataset metadata:
- There are 2 sensors,.
- Each will return 3 channels.
- Hence, we will be having `3` + `3` = **`6`** features.
- Their names are ***"acceleration"*** & ***"angular velocity"*** returned by accelerometer and gyroscope respectively.
- BUTT: There are **2** types of acceleration:
    1. Total acceleration
    2. Body acceleration = Total acceleration - gravity
- Hence, we have `3` + `3` + `3` = **`9`** final features in the dataset.

About the time-series:

- Each measurement was sampled at `50Hz` = 50 measurements/sec
- Taken at the intervals of `2.56` seconds.
- Thus, we have 50 * 2.56 = `128` measurements for each time-series.
- Meaning, **the leanght** of time series corresponding to **each** activity is 128.

Some pre-processing is done already:

- Noise removal
- Low pass filtering (smoothing - EWMA *if you remember* 😉)
- Scaling [-1, 1]

Hence, the time-series is not in a raw version.

      +--------+
      /        /|
     /    T   / |
    +--------+  |
    |        | D|
    |   N    |  +
    |        | /
    |        |/
    +--------+

Where:
- N: Number of observations
- T: Features
- D: What makes the feature? (128 observations)

Till now, this is the visualization of the data till now. And what about **Y**? It is the simple `1D` data encoded in {0, 1, 2, 3, 4, 5, 6}

#### 🤔 Does this look... a bit strange?
Yes, because till now... we have only worked with the **2D** data. Or just NxT data. Now are are dealing with **Multivariate** data.

Before this, we had **Univariate** data, thus we were able to store them in the 2D table. Here still a 2D table will be there, but the data itself has to be treated *differently*.

> Here is the [link](https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones) to the dataset. Have a look if there is any confusion... or anything.

### 🛣 Side track